# RNN을 이용한 네이버 영화 리뷰 감정 분석

Data Download : https://github.com/e9t/nsmc/

- ratings_train.txt
- ratinns_test.txt

### 1. txt Dataset을 csv로 변환

In [1]:
import pandas as pd

columns = ['id', 'text', 'label']
train_data = pd.read_csv('../txt_datasets/ratings_train.txt', sep='\t', names=columns, skiprows=1).dropna() # null data 삭제
test_data = pd.read_csv('../txt_datasets/ratings_test.txt', sep='\t', names=columns, skiprows=1).dropna()

# csv 파일로 변환
train_data.to_csv('../csv_datasets/ratings_train.csv', index=False)
test_data.to_csv('../csv_datasets/ratings_test.csv', index=False)


print(train_data.head())

         id                                               text  label
0   9976970                                아 더빙.. 진짜 짜증나네요 목소리      0
1   3819312                  흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나      1
2  10265843                                  너무재밓었다그래서보는것을추천한다      0
3   9045019                      교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정      0
4   6483659  사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...      1


### 2. 전처리

In [2]:
import torch
from torchtext import data

SEED = 1234
torch.manual_seed(SEED)

- Field를 지정한다. 한글 데이터를 다루므로 토크나이저로 spacy를 쓸 수 없다.

In [3]:
from konlpy.tag import Okt  # 형태소 분석기중 하나인 Okt를 불러온다
Okt = Okt()  # Okt 클래스의 인스턴스 생성. 이 인스턴스를 사용하여 텍스트를 형태소로 분리할 수 있다.

# torchtext에 내장된 데이터셋을 이용하는 게 아니므로, 각 컬럼별로 해당하는 Field를 지정해줘야 한다.
TEXT = data.Field(tokenize=Okt.morphs) # data.Field는 텍스트 데이터를 어떻게 처리할지 정의한다. torkenize 인자로 Okt.morphs를 사용하고 있으므로, 각 문장을 형태소로 분리한다.
LABEL = data.LabelField(dtype = torch.float) # data.LabelField는 label을 어떻게 처리할지 정의한다.

# 위에서 정의한 TEXT와 LABEL은 이후 과정에서 데이터를 전처리하는데 사용된다.

In [4]:
# CSV 파일의 각 컬럼이 어떻게 처리될지를 정의하는 딕셔너리를 생성한다.
# 'text' 컬럼은 TEXT Field를 사용하여 처리되고, 'label' 컬럼은 LABEL 필드를 사용하여 처리된다.
fields = {'text' : ('text', TEXT), 'label' : ('label', LABEL)}

# dictionary 형식은 {csv 컬럼명 : (데이터 컬럼명, Field 이름)}

In [5]:
# TabularDataset 은 데이터를 불러오면서 필드에서 정의했던 토큰화 방법으로 토큰화를 수행한다.
train_data, test_data = data.TabularDataset.splits(
    path = '../csv_datasets',
    train = 'ratings_train.csv',
    test = 'ratings_test.csv',
    format = 'csv',
    fields = fields, 
)

In [6]:
# 불러온 데이터의 형태 확인
vars(train_data[0]), vars(train_data[1]), vars(train_data[2])

({'text': ['아', '더빙', '..', '진짜', '짜증나네요', '목소리'], 'label': '0'},
 {'text': ['흠',
   '...',
   '포스터',
   '보고',
   '초딩',
   '영화',
   '줄',
   '....',
   '오버',
   '연기',
   '조차',
   '가볍지',
   '않구나'],
  'label': '1'},
 {'text': ['너', '무재', '밓었', '다그', '래서', '보는것을', '추천', '한', '다'], 'label': '0'})

In [7]:
# 훈련 데이터의 갯수 확인
print("훈련 데이터 수 : {}".format(len(train_data)))
print("테스트 데이터 수 : {}".format(len(test_data)))

훈련 데이터 수 : 149995
테스트 데이터 수 : 49997


In [8]:
# 데이터에 검증 데이터가 따로 주어져 있지 않으므로 생성해준다.
import random
random.seed(SEED)
train_data, valid_data = train_data.split(random_state=random.seed(SEED))
# torchtext의 split 메서드는 기본적으로 데이터를 7:3 비율로 나눈다.

In [9]:
print("훈련 데이터 수 :{}".format(len(train_data)))
print("검증 데이터 수 :{}".format(len(valid_data)))
print("테스트 데이터 수 :{}".format(len(test_data)))

훈련 데이터 수 :104996
검증 데이터 수 :44999
테스트 데이터 수 :49997


In [10]:
# 총 단어가 몇 개인지 확인 
# Field 객체의 build_vocab는 주어진 데이터셋에 대한 단어장(vocabulary)을 만든다. 이는 각 단어를 고유한 정수 인덱스에 매핑한다.
TEXT.build_vocab(train_data) # 모든 단어를 단어장에 포함시킨다. 

print(len(TEXT.vocab))

85768


In [11]:
# 이 중 최소 두 번 이상 등장하는 단어의 갯수 확인
TEXT.build_vocab(train_data, min_freq = 2) # 최소 두 번 이상 등장하는 단어만 단어장에 포함시킨다.
len(TEXT.vocab)

38586

In [12]:
# 35000개로 단어를 끊는다
MAX_VOCAB_SIZE = 35000

# 단어장의 크기를 35000개로 제한한다. 만약 단어장에 더 많은 단어가 있으면, 빈도 수가 낮은 단어부터 제외된다.
TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE) 
LABEL.build_vocab(train_data) # 라벨에 대해서도 단어장을 만든다. 이진 분류 문제이므로 라벨 단어장의 크기는 2이다.

In [13]:
# TEXT, LABEL 단어장의 갯수 확인

print("TEXT 단어장의 갯수 : {}".format(len(TEXT.vocab)))
print("LABEL 단어장의 갯수 : {}".format(len(LABEL.vocab)))

# <unk>와 <pad> 토큰이 추가되어 있으므로 단어의 갯수가 35,002개이다. 
# <unk> : Unknown을 나타내며, 단어장에 없는 단어를 대체한다. train data에 등장하지 않은 단어가 test data에서 나타나면 해당 단어는 <unk>로 처리된다
# <pad> : Padding을 나타내며, 배치 처리를 위해 모든 문장을 동일한 길이로 만들어야 할 때, 짧은 문장에 패딩을 추가하기 위해 사용된다. 
# 패딩은 실제 의미를 가지지 않는 토큰으로 문장의 길이를 맞춘다. 

TEXT 단어장의 갯수 : 35002
LABEL 단어장의 갯수 : 2


- 최빈 단어들이 어떤 것인지 확인

In [14]:
# 최빈 단어 20개 확인
print(TEXT.vocab.freqs.most_common(20))

[('.', 47281), ('이', 39303), ('영화', 35343), ('의', 21608), ('..', 20398), ('가', 19293), ('에', 18781), ('을', 16252), ('...', 16017), ('도', 15031), ('들', 13325), (',', 12338), ('는', 12315), ('를', 11313), ('은', 11161), ('너무', 7753), ('?', 7739), ('한', 7676), ('다', 7146), ('정말', 6899)]


- 단어와 인덱스 사이 매핑 확인

In [15]:
# itos(integer to string) : 정수 인덱스를 단어로 매핑하는 리스트
# 여기서 처음 10개의 요소를 출력하면, 단어장의 상위 10개 단어를 볼 수 있다.
print(TEXT.vocab.itos[:10]) 

# 단어를 정수로 매핑
# stoi(string to integer) : 단어를 정수 인덱스로 매핑하는 딕셔너리
# LABEL의 경우 레이블(클래스)에 대한 매핑을 나타낸다.
print(LABEL.vocab.stoi)

['<unk>', '<pad>', '.', '이', '영화', '의', '..', '가', '에', '을']
defaultdict(None, {'0': 0, '1': 1})


In [50]:
# BucketIterator를 이용하여 데이터 생성자를 만든다
# 데이터를 배치로 나누고 반복할 수 있는 iterator를 생성하는 과정
BATCH_SIZE = 64

device = 'cuda' if torch.cuda.is_available() else 'cpu'

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits( 
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    device = device,
    sort_key = lambda x: len(x.text),
    sort_within_batch = False,
)



In [51]:
# 생성된 데이터의 크기 확인

next(iter(train_iterator)).text.shape
# [문장의 길이 * 배치 사이즈]

# BucketIterator는 배치 내의 문장을 가능한 같은 길이로 만들기 위해 패딩을 사용한다.
# 따라서 배치 내의 모든 문장은 같은 길이인 65로 만들어진다.
# 만약 실제 문장의 길이가 65보다 작다면, 패딩 토큰이 추가되어 문장의 길이가 65가 된다. 



torch.Size([65, 64])

### 3. 모델 생성

- 바닐라 RNN을 이용하여 모델 생성  

모델의 각 input/output 벡터의 사이즈는 다음과 같다
- text : [sentence length, batch size]
- embedded : [sentence length, batch size, embedding dim]
- output : [sentence length, batch size, hidden dim]
- hidden : [1, batch size, hidden dim]

In [35]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        # input_dim : 단어의 개수(TEXT 사전의 길이)
        # embedding_dim : 임베딩은 입력 차원을 연속 (밀집)벡터 공간으로 변환하는 것. 임베딩 차원은 이 벡터 공간의 크기를 의미
        # 이렇게 변환된 벡터는 다음 레이어 RNN으로 전달된다.
        # hidden_dim : RNN의 은닉 상태의 차원
        # output_dim : 최종 출력 차원
        
        super().__init__()  # 참고 : super(RNN, self).__init__()은 Python2 스타일의 문법. python3에서는 이렇게 해도 됨
        self.embedding = nn.Embedding(input_dim, embedding_dim) # 입력 단어를 연속 (밀집)벡터 공간으로 변환
        self.rnn = nn.RNN(embedding_dim, hidden_dim) # 입력 시퀀스의 각 원소에 대해 hidden state를 Update
        self.fc = nn.Linear(hidden_dim, output_dim) # 최종 출력을 생성
        
    def forward(self, text):
        # 입력 text : [sent_len, batch_size]
        
        embedded = self.embedding(text)
        # 임베딩 계층을 통과한 후의 출력 = [sent_len, batch_size, emb_dim]
        
        output, hidden = self.rnn(embedded) 
        # output : [sent_len, batch_size, hidden_dim]
        # hidden : [1, batch_size, hidden_dim]
        
        assert torch.equal(output[-1, :, :], hidden.squeeze(0))  
        # output의 마지막 슬라이스와 hidden이 동일한지 확인한다.
        # hidden 텐서는 시퀀스의 마지막 시간 단계에서의 hidden state를 담고 있음. hidden.squeeze(0) 는 [batch_size, hidden_dim] 형태
        # output 텐서는 각 시간 단계의 hidden state를 저장함. 따라서 output[-1, :, :]는 시퀀스의 마지막 시간 단계에서의 hidden state를 나타냄
        
        return self.fc(hidden.squeeze(0)) # [batch_size, output_dim]

In [36]:
# hyper parameters
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

In [37]:
# model
model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [38]:
# 모델의 파라미터 수 추출

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print("이 모델은 {}개의 파라미터를 가지고 있다".format(count_parameters(model)))

이 모델은 3592105개의 파라미터를 가지고 있다


### 4. 모델 훈련

In [39]:
import torch.optim as optim

# optimizer
optimizer = optim.SGD(model.parameters(), lr = 0.003)

In [53]:
# loss function : binary cross entropy with logits
# BCEWithLogitLoss() : 임의의 실수를 입력으로 받아서 sigmoid 함수를 취해, 0과 1 사이의 값으로 변환한 뒤 label과 BCE를 계산한다

criterion = nn.BCEWithLogitsLoss()

In [54]:
# 모델과 손실함수를 GPU에 올린다

model = model.to(device)
criterion = criterion.to(device)

In [44]:
# 평가를 위해 임의의 실수를 0과 1 두 정수 중 하나로 변환하는 함수 만들기
def binary_accuracy(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float()
    acc = correct.sum() / len(correct)
    return acc
    

In [57]:
# 모델의 훈련과 평가를 위한 함수 만들기
# model의 output size는 [batch size, output_dim] 인데, output_dim = 1이므로 이 차원을 없애줘야 label과 비교할 수 있다. 따라서 squeeze(1) 적용한다

def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    
    for batch in iterator:
        optimizer.zero_grad()
        predictions = model(batch.text).squeeze(1)
        loss = criterion(predictions, batch.label)
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [46]:
# 평가를 위한 함수는 그래디언트 업데이트를 하지 않아야 하므로 with torch.no_grad(): 구문으로 감싼다

def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.eval()
    
    with torch.no_grad():
        for batch in iterator:
            predictions = model(batch.text).squeeze(1)
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)
            
            epoch_loss += loss.item()
            epoch_acc += acc.item()
            
        return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [47]:
# epoch마다 걸린 훈련 시간을 측정하는 함수 만들기
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
# 실제 모델 훈련, 검증 셋의 손실이 개선되면 모델을 저장하도록 구현

N_EPOCHS = 5
best_valid_loss = float('inf') # 초기에는 무한대로 설정됨

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss :
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'checkpoint/2.1_RNN_Sentiment_Analysis.pt')
        
    print("epoch : {}/{} | time : {}m {}s".format(
        epoch+1, N_EPOCHS, epoch_mins, epoch_secs
    ))
    print("train loss : {}, train acc : {}".format(train_loss, train_acc * 100))
    print("Val_loss : {}, Val_acc : {}".format(valid_loss, valid_acc * 100))

epoch : 1/5 | time : 0m 11s
train loss : 0.6929226821330464, train acc : 50.26025797157009
Val_loss : 0.6883356756615367, Val_acc : 54.45350548252463
epoch : 2/5 | time : 0m 11s
train loss : 0.6929856560202651, train acc : 50.30342271094058
Val_loss : 0.6883635412562977, Val_acc : 54.59491680376232
epoch : 3/5 | time : 0m 10s
train loss : 0.6930777966649266, train acc : 49.93863836575542
Val_loss : 0.6882084793495861, Val_acc : 54.61077009412375
epoch : 4/5 | time : 0m 10s
train loss : 0.6929940632535945, train acc : 50.299190873783004
Val_loss : 0.6881241702728651, Val_acc : 54.635501221161
epoch : 5/5 | time : 0m 10s
train loss : 0.6929318275777107, train acc : 50.02285191956124
Val_loss : 0.6881870889020237, Val_acc : 54.69098772684281


성능이 좋지 않다 -> 6.3 : Multi-layer bi-directional LSTM 을 이용해서 진행